<a href="https://www.kaggle.com/code/ceecreative/real-estate-demand-prediction-baseline-model?scriptVersionId=262404712" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sb

import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/china-real-estate-demand-prediction/sample_submission.csv
/kaggle/input/china-real-estate-demand-prediction/test.csv
/kaggle/input/china-real-estate-demand-prediction/train/city_search_index.csv
/kaggle/input/china-real-estate-demand-prediction/train/land_transactions_nearby_sectors.csv
/kaggle/input/china-real-estate-demand-prediction/train/new_house_transactions_nearby_sectors.csv
/kaggle/input/china-real-estate-demand-prediction/train/city_indexes.csv
/kaggle/input/china-real-estate-demand-prediction/train/pre_owned_house_transactions.csv
/kaggle/input/china-real-estate-demand-prediction/train/new_house_transactions.csv
/kaggle/input/china-real-estate-demand-prediction/train/land_transactions.csv
/kaggle/input/china-real-estate-demand-prediction/train/sector_POI.csv
/kaggle/input/china-real-estate-demand-prediction/train/pre_owned_house_transactions_nearby_sectors.csv


In [2]:
#IMPORT TRAIN DATASETS

city = pd.read_csv('/kaggle/input/china-real-estate-demand-prediction/train/city_search_index.csv')
                   
land_nearby = pd.read_csv('/kaggle/input/china-real-estate-demand-prediction/train/land_transactions_nearby_sectors.csv')

new_house_nearby = pd.read_csv('/kaggle/input/china-real-estate-demand-prediction/train/new_house_transactions_nearby_sectors.csv')

city_index = pd.read_csv('/kaggle/input/china-real-estate-demand-prediction/train/city_indexes.csv')

pre_owned_house = pd.read_csv('/kaggle/input/china-real-estate-demand-prediction/train/pre_owned_house_transactions.csv')

new_house = pd.read_csv('/kaggle/input/china-real-estate-demand-prediction/train/new_house_transactions.csv')

land = pd.read_csv('/kaggle/input/china-real-estate-demand-prediction/train/land_transactions.csv')

sector_roi = pd.read_csv('/kaggle/input/china-real-estate-demand-prediction/train/sector_POI.csv')

pre_owned_house_nearby = pd.read_csv(
    '/kaggle/input/china-real-estate-demand-prediction/train/pre_owned_house_transactions_nearby_sectors.csv')

In [3]:
#IMPORT TEST DATASET

test_df = pd.read_csv('/kaggle/input/china-real-estate-demand-prediction/test.csv')

# 1. Cleaning

In [4]:
#CLEAN TEST DATA

test_df['month'] = test_df['id'].str.split('_').str[0]
test_df['sector'] = test_df['id'].str.split('_').str[1]

In [5]:
#DROP ID COLUMN

test_df = test_df.drop(['id','new_house_transaction_amount'], axis = 1)

In [6]:
#drop columns
drop_columns = ['total_fixed_assets_10k','main_business_taxes_and_surcharges_10k',
                'total_fixed_asset_investment_10k',
                'real_estate_development_investment_completed_10k',    
                'residential_development_investment_completed_10k',             
                'science_expenditure_10k',                                     
                'education_expenditure_10k']

city_index = city_index.drop(drop_columns, axis=1)

In [7]:
pre_owned_house[
'price_pre_owned_house_transactions'] = pre_owned_house[
'price_pre_owned_house_transactions'].where(
    pre_owned_house['num_pre_owned_house_transactions'] != 0, 0)

In [8]:
land_nearby['month'] = pd.to_datetime(land_nearby['month'])
new_house_nearby['month'] = pd.to_datetime(new_house_nearby['month'])
pre_owned_house['month'] = pd.to_datetime(pre_owned_house['month'])
new_house['month'] = pd.to_datetime(new_house['month'])
land['month'] = pd.to_datetime(land['month'])
pre_owned_house_nearby['month'] = pd.to_datetime(pre_owned_house_nearby['month']) 

# test data
test_df['month'] = pd.to_datetime(test_df['month']) 

In [9]:
new_house['amount_new_house_transactions'] = (new_house[
'area_new_house_transactions'] * new_house['price_new_house_transactions']) /10000

In [10]:
print('city shape:',city.shape)
print('land_nearby :',land_nearby.shape)
print('new_house_nearby :',new_house_nearby.shape)
print('city_index :',city_index.shape)
print('pre_owned_house',pre_owned_house.shape)
print('new_house :',new_house.shape)
print('land :',land.shape)
print('sector_roi :',sector_roi.shape)
print('pre_owned_house_nearby :',pre_owned_house_nearby.shape) 

city shape: (4020, 4)
land_nearby : (5025, 6)
new_house_nearby : (5360, 11)
city_index : (7, 67)
pre_owned_house (5360, 6)
new_house : (5433, 11)
land : (5896, 6)
sector_roi : (86, 142)
pre_owned_house_nearby : (5427, 6)


# 2. Feature Engineering

In [11]:
unique_sectors = sorted(new_house['sector'].unique())

In [12]:
unique_dates = pd.date_range(start = new_house['month'].min(), 
                             end = new_house['month'].max(), freq='MS')

In [13]:
all_combos = pd.MultiIndex.from_product([unique_dates,unique_sectors],
                           names = ['month', 'sector'])

In [14]:
master_df = pd.DataFrame(index=all_combos).reset_index()

In [15]:
master_df = master_df.merge(land_nearby, on = ['month', 'sector'], how = 'left')
master_df = master_df.merge(new_house_nearby, on = ['month', 'sector'], how = 'left')
master_df = master_df.merge(pre_owned_house, on = ['month', 'sector'], how = 'left')
master_df = master_df.merge(new_house, on = ['month', 'sector'], how = 'left')
master_df = master_df.merge(land, on = ['month', 'sector'], how = 'left')
master_df = master_df.merge(pre_owned_house_nearby, on = ['month', 'sector'], how = 'left')

In [16]:
master_df = master_df.merge(sector_roi, on = ['sector'], how = 'left')

In [17]:
master_df.insert(1, 'year', master_df['month'].dt.year)

In [18]:
city_index_unique = city_index.groupby('city_indicator_data_year').mean().reset_index()

In [19]:
master_df = master_df.merge(city_index_unique, left_on = 'year', 
                right_on = 'city_indicator_data_year', how = 'left' )

In [20]:
master_df = pd.concat([master_df,test_df],axis=0)

In [21]:
master_df = master_df.reset_index(drop=True)

In [22]:
master_df = master_df.fillna(0)

In [23]:
empty_columns = []

for col in master_df.columns:
    if master_df[col].max() == 0:
        empty_columns.append(col)

#DROP THE EMPTY COLUMNS
master_df = master_df.drop(empty_columns, axis = 1)

In [24]:
from sklearn.preprocessing import OneHotEncoder

In [25]:
# Create encoder with unknown handling
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit and transform
result = encoder.fit_transform(master_df[['year']])

# Convert to DataFrame with readable column names
encoded_df = pd.DataFrame(result, columns=encoder.get_feature_names_out(
    ['year']), index=master_df.index) 

#drop year_0. its empty
encoded_df = encoded_df.drop('year_0.0',axis=1)

master_df = pd.concat([master_df,encoded_df], axis=1)

In [26]:
master_df.shape

(7517, 243)

In [27]:
target_col = 'amount_new_house_transactions'

columns = [target_col, 'area_new_house_transactions', 
           'price_new_house_transactions',
          'area_pre_owned_house_transactions',
       'amount_pre_owned_house_transactions',
       'num_pre_owned_house_transactions',
       'price_pre_owned_house_transactions'] 

In [28]:
lags = [1,3,6]

for col in columns:
    for lag in lags:
        master_df[f"{col}_lag_{lag}"] = master_df.groupby('sector')[col].shift(lag).fillna(0)

In [29]:
windows = [3,6,12]

for col in columns:
    for window in windows:
        master_df[f"{col}_roll_{window}"] = master_df.groupby(
            'sector')[col].transform(lambda x:x.rolling(window).mean()).fillna(0)

In [30]:
cut_off_date = pd.to_datetime('2024-07-01')

#DIVIDE TRAIN AND TEST DATA\

new_train = master_df[master_df['month'] <= cut_off_date]
new_test = master_df[master_df['month'] > cut_off_date]

# 4. Modelling

In [31]:
train_df = new_train.copy()

In [32]:
#FEATURES
X_train = train_df.drop([target_col,'month','year',
             'sector','city_indicator_data_year'],axis=1)

X_test = new_test.drop([target_col,'month','year',
             'sector','city_indicator_data_year'],axis=1)

#TARGET
y_train = train_df[target_col]

In [33]:
#import models
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#metrics
from sklearn.metrics import mean_absolute_percentage_error

In [34]:
# performing preprocessing part

sc = StandardScaler()

X_train_scaled = sc.fit_transform(X_train)

In [35]:
from sklearn.model_selection import TimeSeriesSplit

# Create a time series cross-validator with 3 splits
tscv = TimeSeriesSplit(n_splits=3)

for train_index, test_index in tscv.split(X_train_scaled):
    # Split data into train and test based on indices
    X_train_fold, X_val_fold = X_train_scaled[train_index], X_train_scaled[test_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[test_index]
    
    # train model on past data (X_train, y_train)
    xg = XGBRegressor()
    xg.fit(X_train_fold,y_train_fold)
    
    # And evaluate it on future data (X_test, y_test)
    model = xg.predict(X_val_fold)

    #check model performance
    error = np.sqrt(mean_absolute_percentage_error(model,y_val_fold))

    #print results
    print(f"MAPE : {error : .3f}")

MAPE :  0.630
MAPE :  0.517
MAPE :  0.586


# 6. Submission

In [36]:
predictions = xg.predict(X_test)

In [37]:
new_test['new_house_transaction_amount'] = predictions

In [38]:
new_test['id'] = new_test['month'].dt.strftime("%Y %b") + '_' + new_test['sector'] 

In [39]:
submission_df = new_test[['id','new_house_transaction_amount']]

In [40]:
submission_df.to_csv('/kaggle/working/submission.csv', index=False)

In [41]:
MAPE :  0.630
MAPE :  0.517
MAPE :  0.586